In [1]:
#@title Setup (Do NOT modify)
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/final
!pip install openai-clip

Mounted at /content/drive
/content/drive/MyDrive/final
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 7.9 MB/s eta 0:00:00
  Created wheel for openai-clip: filename=openai_clip-1.0.1-py3-none-any.whl size=1368605 sha256=524267c639b831a15efabc59d67b5090c8d046a7795b65b049272bd84294d4f0
  Stored in directory: /root/.cache/pip/wheels/08/77/8e/8d2f862df6bf7fb4e2007062d2cbaeae49862ec7b56d041229
Successfully built openai-clip


In [ ]:
#@title Data preprocessing (Train)
!python preproc_datasets_celeba_zip_train.py --source=./multimodal_celeba_hq.zip --dest train_data_6cap.zip --emb_dim 512 --transform=center-crop --width=256 --height=256

In [ ]:
#@title Data preprocessing (Test)
!python preproc_datasets_celeba_zip_test.py --source=./multimodal_celeba_hq.zip --dest test_data_6cap.zip --emb_dim 512 --transform=center-crop --width=256 --height=256

In [2]:
#@title Train

train_data = "train_data_4cap.zip"#@param {"type": "string"}
batch_size = 24 #@param {"type": "integer"}
num_epochs = 5 #@param {"type": "integer"}
learning_rate = 0.0001 #@param {"type": "number"}
report_interval = 50 #@param {"type": "integer"}
noise_dim = 100 #@param {"type": "integer"}
projection_dim = 128 #@param {"type": "integer"}
clip_embedding_dim = 512 #@param {"type": "integer"}
checkpoint_path = "model_exp1" #@param {"type": "string"}
result_path = "images_exp1" #@param {"type": "string"}
use_uncond_loss = True #@param {"type": "boolean"}
use_contrastive_loss = True #@param {"type": "boolean"}
num_stage = 3 #@param {"type": "integer"}
resume_checkpoint_path = "None" #@param {"type": "string"}
resume_epoch = -1 #@param {"type": "integer"}

test_cmd = f'''python main.py \
    --train_data "{train_data}" \
    --batch_size {batch_size} \
    --num_epochs {num_epochs} \
    --learning_rate {learning_rate} \
    --report_interval {report_interval} \
    --noise_dim {noise_dim} \
    --projection_dim {projection_dim} \
    --clip_embedding_dim {clip_embedding_dim} \
    --checkpoint_path "{checkpoint_path}" \
    --result_path "{result_path}" \
    --num_stage {num_stage} \
    --resume_epoch {resume_epoch} \
    '''
if use_uncond_loss:
    test_cmd += "--use_uncond_loss "
if use_contrastive_loss:
    test_cmd += "--use_contrastive_loss "
if resume_checkpoint_path != "None":
    test_cmd += f'''--resume_checkpoint_path "{resume_checkpoint_path}"'''

with open('./train_script.sh', 'w') as file:
    file.write(test_cmd)

!bash train_script.sh

Loading dataset
finish
100%|███████████████████████████████████████| 338M/338M [00:08<00:00, 41.1MiB/s]
Epoch: 0 start
    Iteration 50 	 d_loss: 1109.3487, g_loss: 977.6000
    Iteration 100 	 d_loss: 1077.1721, g_loss: 953.7432
    Iteration 150 	 d_loss: 1055.3281, g_loss: 944.4468
    Iteration 200 	 d_loss: 1011.0840, g_loss: 933.9455
    Iteration 250 	 d_loss: 993.6214, g_loss: 909.4641
    Iteration 300 	 d_loss: 1000.2929, g_loss: 900.2933
    Iteration 350 	 d_loss: 961.2294, g_loss: 881.8640
    Iteration 400 	 d_loss: 966.3742, g_loss: 884.9607
    Iteration 450 	 d_loss: 941.8362, g_loss: 862.2460
    Iteration 500 	 d_loss: 922.0736, g_loss: 859.9550
    Iteration 550 	 d_loss: 924.1231, g_loss: 857.3235
    Iteration 600 	 d_loss: 916.1769, g_loss: 852.6349
    Iteration 650 	 d_loss: 897.8267, g_loss: 838.3499
    Iteration 700 	 d_loss: 919.0027, g_loss: 854.3065
    Iteration 750 	 d_loss: 909.9120, g_loss: 853.2668
    Iteration 800 	 d_loss: 909.0764, g_loss: 857.90

In [5]:
#@title Test (Generate image)

prompt = "The woman is young and has blond hair, and arched eyebrows."#@param {"type": "string"}
load_epoch = 10 #@param {"type": "integer"}
checkpoint_path = "model_exp1" #@param {"type": "string"}

test_cmd = f'''python generate_image.py \
    --prompt "{prompt}" \
    --load_epoch {load_epoch} \
    --checkpoint_path "{checkpoint_path}"
    '''

with open('./test_script.sh', 'w') as file:
    file.write(test_cmd)

!bash test_script.sh

Cannot load checkpoint; check the directory
